# Make dataset with specific mass balance and climatic and topographic input variables


## Import libraries

In [1]:
import numpy as np
import pandas as pd
import re # regular expression
from pathlib import Path # for reincursive folders
import xarray as xr # To open .nc data as array
import geopandas as gpd
import rasterio
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
root_dir = 'C:/Users/david/Documents/Thesis/Data/'

## Ice thickness

In [31]:
%%time
ice_thickness_df = pd.DataFrame()
ice_thickness = []
rgi_ids = []

root_dir_ice = 'C:/Users/david/Documents/Thesis/Data/IceThickness/'
for filename in Path(root_dir_ice).rglob('*.tif'):
    filename = str(filename)
    rgi_id = re.findall(r"RGI60-[0-9]*\.[0-9]{5}",filename) # RegEx
    rgi_id = ''.join(rgi_id) # from list to string
    with rasterio.open(filename) as src:
        array = src.read(1)
    array[array==0] = np.nan
    array = array[~numpy.isnan(array)]
    mean = array.mean()
    ice_thickness.append(mean)
    rgi_ids.append(rgi_id)

ice_thickness_df['rgi_id'] = rgi_ids
ice_thickness_df['ice_thickness'] = ice_thickness
ice_thickness_df
# zie consensus estimate Farinotti

CPU times: total: 24min 32s
Wall time: 32min 12s


,rgi_id,ice_thickness
0,RGI60-13.00001,33.668629
1,RGI60-13.00002,39.834972
2,RGI60-13.00003,14.957049
3,RGI60-13.00004,23.991674
4,RGI60-13.00005,20.455612
...,...,...
95529,RGI60-15.13115,36.147495
95530,RGI60-15.13116,90.726250
95531,RGI60-15.13117,110.223427
95532,RGI60-15.13118,14.799946


## Debris thickness

In [2]:
# Only glaciers larger than 0.4km2 are included in dataset.
debris_thickness = pd.read_csv(root_dir + 'Debris_thickness/rgi60_points_gt04km2_debrisstats.csv')
debris_thickness = debris_thickness.rename(columns={'RGIId':'rgi_id'})

In [4]:
debris_thickness['debris_area_ela_p'] = (debris_thickness['deba_ela']/debris_thickness['tota_ela'])*100
debris_thickness = debris_thickness[['rgi_id', 'debris_area_ela_p','vol_ela_p']]
debris_thickness = debris_thickness.rename(columns={'vol_ela_p':'debris_vol_ela_p'})
debris_thickness

,rgi_id,debris_area_ela_p,debris_vol_ela_p
0,RGI60-13.00001,3.636364,0.156586
1,RGI60-13.00014,6.293706,0.065130
2,RGI60-13.00015,6.097561,2.135240
3,RGI60-13.00017,9.604520,0.896338
4,RGI60-13.00020,7.500000,0.499605
...,...,...,...
33582,RGI60-15.13097,0.080064,0.003631
33583,RGI60-15.13104,20.432626,13.621900
33584,RGI60-15.13115,3.452528,0.344002
33585,RGI60-15.13116,6.750241,0.523140


## RGI data and surging data

In [3]:
rgi13 = gpd.read_file(root_dir + 'RGI_data/13_rgi60_CentralAsia.shp')
rgi14 = gpd.read_file(root_dir + 'RGI_data/14_rgi60_SouthAsiaWest.shp')
rgi15 = gpd.read_file(root_dir + 'RGI_data/15_rgi60_SouthAsiaEast.shp')
rgi = pd.concat([rgi13,rgi14,rgi15], ignore_index=True)
rgi = rgi.rename(columns={"RGIId": "rgi_id"})
rgi = rgi[['rgi_id','Slope','Aspect','Lmax','Area','Zmax','Zmin','Zmed','O2Region']]
rgi_df = pd.DataFrame(rgi)


In [4]:
rgi_df_2km = rgi_df[rgi_df['Area']>2]
rgi_df_2km.to_csv('rgi_df_2km.csv')

# Surging data:
surging_df = pd.read_csv(root_dir + '/surging.csv')
surging_df['surging'] = 1
surging_df = surging_df.rename(columns={'RGIId':'rgi_id'})
rgi_surging = rgi_df_2km.merge(surging_df, on = 'rgi_id', how = 'outer')
rgi_surging = rgi_surging.fillna(0)
rgi_surging = rgi_surging[rgi_surging['Area']>2]
rgi_surging

,rgi_id,Slope,Aspect,Lmax,Area,Zmax,Zmin,Zmed,O2Region,surging
0,RGI60-13.00062,21.5,358.0,1532.0,2.222,6011.0,5417.0,5839.0,5,0.0
1,RGI60-13.00093,25.6,50.0,1219.0,2.019,6095.0,5432.0,5795.0,5,0.0
2,RGI60-13.00130,26.4,346.0,2145.0,2.439,6308.0,5428.0,5864.0,5,0.0
3,RGI60-13.00135,16.4,293.0,4204.0,7.489,6186.0,5357.0,5917.0,5,0.0
4,RGI60-13.00137,20.6,143.0,2668.0,2.224,6310.0,5756.0,6049.0,5,0.0
...,...,...,...,...,...,...,...,...,...,...
8094,RGI60-15.13092,21.2,165.0,2761.0,4.619,5524.0,4583.0,5152.0,3,0.0
8095,RGI60-15.13097,8.8,20.0,4916.0,3.616,5908.0,5234.0,5658.0,1,0.0
8096,RGI60-15.13104,12.2,341.0,11899.0,22.704,6752.0,5235.0,5834.0,1,0.0
8097,RGI60-15.13116,8.9,330.0,5779.0,7.427,6232.0,5266.0,5633.0,1,0.0


## Glacial lakes

In [14]:
glacial_lakes = gpd.read_file(root_dir + 'supraglacial_lakes/Glaciers_Intersect_lakes.shp')
glacial_lakes = pd.DataFrame(glacial_lakes)
glacial_lakes = glacial_lakes[['RGIId']]
glacial_lakes = glacial_lakes.rename(columns = {'RGIId':'rgi_id'})
glacial_lakes['glacial_lake'] = 1
glacial_lakes

,rgi_id,glacial_lake
0,RGI60-13.00001,1
1,RGI60-13.00606,1
2,RGI60-13.00643,1
3,RGI60-13.00713,1
4,RGI60-13.00731,1
...,...,...
2603,RGI60-15.13033,1
2604,RGI60-15.13044,1
2605,RGI60-15.13104,1
2606,RGI60-15.13111,1


## Wrangle climate_historical.nc data

In [12]:
%%time
# For regions 13, 14 and 15
climate_historical = pd.DataFrame()
#root_dir = 'C:/Users/david/Documents/Thesis/Data/'
for filename in Path(root_dir).rglob('**/climate_historical.nc'):
    filename = str(filename)
    rgi_id = re.findall(r"RGI60-[0-9]*\.[0-9]{5}",filename) # RegEx
    rgi_id = ''.join(rgi_id) # from list to string
    array = xr.open_dataset(filename)
    df = array.to_dataframe()
    df = df.resample('10A').agg({'prcp':'sum', 'temp':'mean'}) # uitleggen
    df['rgi_id'] = rgi_id
    df = df.tail(2) # only 2009 and 2019
    climate_historical = climate_historical.append(df)
    
climate_historical

CPU times: total: 37min 43s
Wall time: 46min 25s


,prcp,temp,rgi_id
time,,,
2009-12-31,2720.994385,-10.620624,RGI60-13.00001
2019-12-31,2697.349854,-9.851106,RGI60-13.00001
2009-12-31,2471.323242,-14.763111,RGI60-13.00002
2019-12-31,2483.284668,-14.639121,RGI60-13.00002
2009-12-31,2471.323242,-14.763111,RGI60-13.00003
...,...,...,...
2019-12-31,5144.911133,-4.920659,RGI60-15.13117
2009-12-31,4942.907227,-5.594553,RGI60-15.13118
2019-12-31,5144.911133,-4.920659,RGI60-15.13118


In [272]:
%%time
# For regions 13, 14 and 15
climate_historical_1980_2000 = pd.DataFrame()
#root_dir = 'C:/Users/david/Documents/Thesis/Data/'
for filename in Path(root_dir).rglob('**/climate_historical.nc'):
    filename = str(filename)
    rgi_id = re.findall(r"RGI60-[0-9]*\.[0-9]{5}",filename) # RegEx
    rgi_id = ''.join(rgi_id) # from list to string
    array = xr.open_dataset(filename)
    df = array.to_dataframe()
    df = df.resample('10A').agg({'prcp':'sum', 'temp':'mean'}) # uitleggen
    df['rgi_id'] = rgi_id
    df = df.iloc[1:3] # between 1980 and 2000
    climate_historical_1980_2000 = climate_historical_1980_2000.append(df)
    
climate_historical_1980_2000

CPU times: total: 53min 46s
Wall time: 1h 3min 58s


,prcp,temp,rgi_id
time,,,
1989-12-31,2403.083252,-11.452559,RGI60-13.00001
1999-12-31,2530.919189,-11.293782,RGI60-13.00001
1989-12-31,2515.235107,-16.065098,RGI60-13.00002
1999-12-31,2409.958984,-15.468081,RGI60-13.00002
1989-12-31,2515.235107,-16.065098,RGI60-13.00003
...,...,...,...
1999-12-31,4533.192871,-5.685079,RGI60-15.13117
1989-12-31,4434.066406,-6.334142,RGI60-15.13118
1999-12-31,4533.192871,-5.685079,RGI60-15.13118


In [273]:
climate_historical_1980_2000.to_csv('climate_historical_1980_2000.csv')

In [13]:
climate_historical.to_csv('climate_historical.csv')

## Merge Hugonnet and climate_historical

In [5]:
# Remove 2000 - 2020 time period and rename rgiid to rgi_id
hugonnet = pd.read_csv(root_dir + 'hugonnet_2021_ds_rgi60_pergla_rates_10_20_worldwide.csv')
hugonnet = hugonnet[hugonnet['period'].str.contains("2000-01-01_2020-01-01")==False]
hugonnet = hugonnet.rename(columns={'rgiid':'rgi_id'})
hugonnet = hugonnet.drop_duplicates().reset_index(drop=True)
hugonnet

,rgi_id,period,area,dhdt,err_dhdt,dvoldt,err_dvoldt,dmdt,err_dmdt,dmdtda,err_dmdtda,perc_area_meas,perc_area_res,valid_obs,valid_obs_py,reg
0,RGI60-01.00001,2010-01-01_2020-01-01,360000.0,-0.0556,0.4645,-20003.0,167248.0,-0.000017,0.000142,-0.0472,0.3949,1.000,1.000,18.30,6.32,1
1,RGI60-01.00001,2000-01-01_2010-01-01,360000.0,0.0255,0.5059,9175.0,182132.0,0.000008,0.000155,0.0217,0.4300,1.000,1.000,8.11,4.78,1
2,RGI60-01.00002,2010-01-01_2020-01-01,558000.0,-0.3410,0.3234,-190257.0,181714.0,-0.000162,0.000155,-0.2898,0.2794,1.000,1.000,15.18,7.95,1
3,RGI60-01.00002,2000-01-01_2010-01-01,558000.0,-0.1980,0.3267,-110465.0,182728.0,-0.000094,0.000155,-0.1683,0.2792,1.000,1.000,11.04,8.28,1
4,RGI60-01.00003,2010-01-01_2020-01-01,1685000.0,-1.2268,0.3275,-2067231.0,563356.0,-0.001757,0.000495,-1.0428,0.2991,1.000,1.000,10.06,6.87,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436255,RGI60-19.02750,2010-01-01_2020-01-01,4118000.0,0.0964,1.4274,396778.0,5878013.0,0.000337,0.004996,0.0819,1.2133,0.727,0.734,1.10,0.84,19
436256,RGI60-19.02751,2010-01-01_2020-01-01,11000.0,-2.1904,1.5831,-24095.0,26564.0,-0.000020,0.000023,-1.8619,2.5755,1.000,1.000,4.00,4.00,19
436257,RGI60-19.02751,2000-01-01_2010-01-01,11000.0,-2.1317,1.7586,-23449.0,27483.0,-0.000020,0.000023,-1.8119,2.6080,1.000,1.000,0.00,0.00,19
436258,RGI60-19.02752,2000-01-01_2010-01-01,528000.0,0.1427,0.6371,75344.0,336536.0,0.000064,0.000286,0.1213,0.5421,0.981,0.981,2.74,1.70,19


In [6]:
# Load climate_historical data and make same format as Hugonnet
climate_historical = pd.read_csv(root_dir + 'climate_historical.csv')
climate_historical['time'] = climate_historical['time'].replace({'2009-12-31':'2000-01-01_2010-01-01', '2019-12-31':'2010-01-01_2020-01-01'})
climate_historical = climate_historical.rename(columns={"time": "period"})
climate_historical['prcp'] = climate_historical['prcp'].div(10)
climate_historical

,period,prcp,temp,rgi_id
0,2000-01-01_2010-01-01,272.09944,-10.620624,RGI60-13.00001
1,2010-01-01_2020-01-01,269.73499,-9.851106,RGI60-13.00001
2,2000-01-01_2010-01-01,247.13232,-14.763111,RGI60-13.00002
3,2010-01-01_2020-01-01,248.32847,-14.639121,RGI60-13.00002
4,2000-01-01_2010-01-01,247.13232,-14.763111,RGI60-13.00003
...,...,...,...,...
191067,2010-01-01_2020-01-01,514.49110,-4.920659,RGI60-15.13117
191068,2000-01-01_2010-01-01,494.29070,-5.594553,RGI60-15.13118
191069,2010-01-01_2020-01-01,514.49110,-4.920659,RGI60-15.13118
191070,2000-01-01_2010-01-01,543.18496,-6.466131,RGI60-15.13119


In [7]:
# Join climate_historical and Hugonnet on rgi_id and period
climate_historical_hugo = pd.merge(hugonnet, climate_historical, on=['rgi_id','period'])
# Make subset of merge_climate_hugo_s dataframe. Not include area, because area is same as RGI area
climate_historical_hugo = climate_historical_hugo[['rgi_id','period','dmdtda','prcp','temp','dvoldt']]
climate_historical_hugo

,rgi_id,period,dmdtda,prcp,temp,dvoldt
0,RGI60-13.00001,2000-01-01_2010-01-01,0.0789,272.09944,-10.620624,40091.0
1,RGI60-13.00001,2010-01-01_2020-01-01,-0.3219,269.73499,-9.851106,-163594.0
2,RGI60-13.00002,2000-01-01_2010-01-01,0.0855,247.13232,-14.763111,36902.0
3,RGI60-13.00002,2010-01-01_2020-01-01,-0.0314,248.32847,-14.639121,-13542.0
4,RGI60-13.00003,2000-01-01_2010-01-01,0.4173,247.13232,-14.763111,34369.0
...,...,...,...,...,...,...
191067,RGI60-15.13117,2010-01-01_2020-01-01,-0.5364,514.49110,-4.920659,-4283279.0
191068,RGI60-15.13118,2010-01-01_2020-01-01,-0.8078,514.49110,-4.920659,-40866.0
191069,RGI60-15.13118,2000-01-01_2010-01-01,-0.8557,494.29070,-5.594553,-43290.0
191070,RGI60-15.13119,2000-01-01_2010-01-01,-0.6671,543.18496,-6.466131,-73775.0


## Load climate_statistics.csv data and make subset of variables

In [8]:
climate13 = pd.read_csv(root_dir + 'climate_statistics_13.csv')
climate14 = pd.read_csv(root_dir + 'climate_statistics_14.csv')
climate15 = pd.read_csv(root_dir + 'climate_statistics_15.csv')

climate13_14 = climate13.append(climate14)
climate = climate13_14.append(climate15)
climate_s = climate[['rgi_id','rgi_area_km2','tstar_aar','tstar_ela_h']]
climate_s

,rgi_id,rgi_area_km2,tstar_aar,tstar_ela_h
0,RGI60-13.00001,0.432,0.492122,5764.161635
1,RGI60-13.00002,0.367,0.533972,6048.410212
2,RGI60-13.00003,0.070,0.480691,5756.915813
3,RGI60-13.00004,0.255,0.497794,5762.474899
4,RGI60-13.00005,0.261,0.547779,5753.550930
...,...,...,...,...
13114,RGI60-15.13115,1.356,0.518153,5647.349262
13115,RGI60-15.13116,7.427,0.537703,5613.211517
13116,RGI60-15.13117,6.788,0.580946,5977.445401
13117,RGI60-15.13118,0.043,0.465043,5994.382197


## Merge climate_historical, Hugonnet, climate_statistics.csv data

In [9]:
climate_hugo_climate_glacier = pd.merge(climate_historical_hugo, climate_s, left_on='rgi_id', right_on='rgi_id', how='left')
climate_hugo_climate_glacier = climate_hugo_climate_glacier.drop_duplicates().reset_index(drop=True)
climate_hugo_climate_glacier


,rgi_id,period,dmdtda,prcp,temp,dvoldt,rgi_area_km2,tstar_aar,tstar_ela_h
0,RGI60-13.00001,2000-01-01_2010-01-01,0.0789,272.09944,-10.620624,40091.0,0.432,0.492122,5764.161635
1,RGI60-13.00001,2010-01-01_2020-01-01,-0.3219,269.73499,-9.851106,-163594.0,0.432,0.492122,5764.161635
2,RGI60-13.00002,2000-01-01_2010-01-01,0.0855,247.13232,-14.763111,36902.0,0.367,0.533972,6048.410212
3,RGI60-13.00002,2010-01-01_2020-01-01,-0.0314,248.32847,-14.639121,-13542.0,0.367,0.533972,6048.410212
4,RGI60-13.00003,2000-01-01_2010-01-01,0.4173,247.13232,-14.763111,34369.0,0.070,0.480691,5756.915813
...,...,...,...,...,...,...,...,...,...
191067,RGI60-15.13117,2010-01-01_2020-01-01,-0.5364,514.49110,-4.920659,-4283279.0,6.788,0.580946,5977.445401
191068,RGI60-15.13118,2010-01-01_2020-01-01,-0.8078,514.49110,-4.920659,-40866.0,0.043,0.465043,5994.382197
191069,RGI60-15.13118,2000-01-01_2010-01-01,-0.8557,494.29070,-5.594553,-43290.0,0.043,0.465043,5994.382197
191070,RGI60-15.13119,2000-01-01_2010-01-01,-0.6671,543.18496,-6.466131,-73775.0,0.094,0.571124,5834.455509


## Merge ice thickness

In [10]:
ice_thickness_df = pd.read_csv(root_dir + 'ice_thickness_df.csv')

In [11]:
df = pd.merge(climate_hugo_climate_glacier, ice_thickness_df, left_on='rgi_id', right_on='rgi_id')
df = df.rename({'ice_thickness': 'avg_ice_thickness'}, axis=1)  
df['ice_volume'] = df['rgi_area_km2'] * (df['avg_ice_thickness']/1000)
df['ice_volume']
df = df.drop_duplicates().reset_index(drop=True)

## Merge glacial lakes

In [15]:
df_lakes = df.merge(glacial_lakes, on = 'rgi_id', how = 'outer')
df_lakes = df_lakes[['rgi_id','glacial_lake']].fillna(0)
df_lakes

,rgi_id,glacial_lake
0,RGI60-13.00001,1.0
1,RGI60-13.00001,1.0
2,RGI60-13.00002,0.0
3,RGI60-13.00002,0.0
4,RGI60-13.00003,0.0
...,...,...
191859,RGI60-15.13117,0.0
191860,RGI60-15.13118,0.0
191861,RGI60-15.13118,0.0
191862,RGI60-15.13119,0.0


In [16]:
df = pd.merge(df, df_lakes, left_on='rgi_id', right_on='rgi_id')
df = df.drop_duplicates().reset_index(drop=True)

## Merge RGI and surging data

In [17]:
df = df.merge(rgi_surging, on = 'rgi_id', how = 'inner')
df = df.drop_duplicates().reset_index(drop=True)

## Merge debris thickness

In [18]:
# Only glaciers larger than 0.4 km2 are in debris thickness dataset
df = df[df['Area']>0.4]
df = df.merge(debris_thickness, on = 'rgi_id', how = 'outer')
df = df.drop_duplicates().reset_index(drop=True)

In [19]:
df = df.drop(['rgi_area_km2','Unnamed: 0'], axis=1)

In [20]:
df = df[df['Area'] >2]
df.reset_index(drop=True, inplace=True)

## Merge velocity

In [21]:
velocity = pd.read_csv(root_dir + 'Velocity/glacier_velocity.csv')
velocity = velocity.rename(columns={"RGIId": "rgi_id",'_mean':'velocity_mean'})
velocity = velocity.dropna(how='any',axis=0) 
df = df.merge(velocity, on = 'rgi_id', how = 'outer')
df = df.drop_duplicates().reset_index(drop=True)

In [22]:
df.columns

Index(['rgi_id', 'period', 'dmdtda', 'prcp', 'temp', 'dvoldt', 'tstar_aar',
       'tstar_ela_h', 'avg_ice_thickness', 'ice_volume', 'glacial_lake',
       'Slope', 'Aspect', 'Lmax', 'Area', 'Zmax', 'Zmin', 'Zmed', 'O2Region',
       'surging', 'lon', 'lat', 'tot_area', 'deb_area', 'cln_area', 'tot_vol',
       'deb_vol', 'cln_vol', 'tota_ela', 'deba_ela', 'totv_ela', 'deb_v_ela',
       'area_perc', 'vol_perc', 'vol_ela_p', 'velocity_mean'],
      dtype='object')

## Change periods into binary value

In [24]:
df.loc[df["period"] == "2000-01-01_2010-01-01", "period"] = 0
df.loc[df["period"] == "2010-01-01_2020-01-01", "period"] = 1

## Make column with temperature change

In [31]:
ids = df['rgi_id']
ids = ids.unique()
empty_list_temp_diff = []
empty_list_id = []
empty_list_temp_mean = []

for i in range(len(ids)):
    
    sel2000 =  df[(df['rgi_id'] == ids[i]) & (df['period'] == 0)]
    sel2010 =  df[(df['rgi_id'] == ids[i]) & (df['period'] == 1)]
    
    value_temp_diff = sel2010['temp'].iloc[0] - sel2000['temp'].iloc[0]#negative value is afgenomen temp
    value_temp_mean = (sel2010['temp'].iloc[0] + sel2000['temp'].iloc[0])/2
    
    id_take = ids[i]
    
    empty_list_temp_diff.append(value_temp_diff)
    empty_list_id.append(id_take)
    empty_list_temp_mean.append(value_temp_mean)
    
d = {'rgi_id':empty_list_id,'temp_diff_2000_2020':empty_list_temp_diff,'temp_mean_2000_2020':empty_list_temp_mean}
temp_20y_df = pd.DataFrame(d)

In [94]:
temp_20y_df

,rgi_id,temp_diff_2000_2020,temp_mean_2000_2020
0,RGI60-13.00062,0.152752,-13.178201
1,RGI60-13.00093,0.123990,-14.701116
2,RGI60-13.00130,0.123990,-14.701116
3,RGI60-13.00135,0.123990,-14.701116
4,RGI60-13.00137,0.123990,-14.701116
...,...,...,...
8094,RGI60-15.13092,0.624130,-2.584252
8095,RGI60-15.13097,0.146501,-9.092999
8096,RGI60-15.13104,0.073012,-10.079212
8097,RGI60-15.13116,0.315175,-8.041431


In [95]:
df = df.merge(temp_20y_df, on = 'rgi_id', how= 'outer')

## Make column with precipitation change

In [35]:
ids = df['rgi_id']
ids = ids.unique()
empty_list_prcp_diff = []
empty_list_id = []
empty_list_prcp_mean = []

for i in range(len(ids)):
    
    sel2000 =  df[(df['rgi_id'] == ids[i]) & (df['period'] == 0)]
    sel2010 =  df[(df['rgi_id'] == ids[i]) & (df['period'] == 1)]
 
    value_prcp_diff = sel2010['prcp'].iloc[0] - sel2000['prcp'].iloc[0]#negative value is afgenomen temp
    value_prcp_mean = (sel2010['prcp'].iloc[0] + sel2000['prcp'].iloc[0])/2
    
    id_take = ids[i]
    empty_list_prcp_diff.append(value_prcp_diff)
    empty_list_id.append(id_take)
    empty_list_prcp_mean.append(value_prcp_mean)
    
d = {'rgi_id':empty_list_id,'prcp_diff_2000_2020':empty_list_prcp_diff,'prcp_mean_2000_2020':empty_list_prcp_mean}
prcp_20y_df = pd.DataFrame(d)

In [97]:
df = df.merge(prcp_20y_df, on = 'rgi_id', how= 'outer')

In [98]:
df2000_2020 = df

## Make column with mean temperature and temperature change 1980 - 2000

In [100]:
# laad in data.
rgi_df_2km = rgi_df[rgi_df['Area']>2]
rgi_df_2km = rgi_df_2km[['rgi_id','Area']]
climate_historical_1980_2000 = pd.read_csv(root_dir + 'climate_historical_1980_2000.csv')

In [101]:
climate_historical_1980_2000['time'] = climate_historical_1980_2000['time'].replace({'1989-12-31':'1980-01-01_1990-01-01', '1999-12-31':'1990-01-01_2000-01-01'})

In [102]:
climate_historical_1980_2000 = climate_historical_1980_2000.merge(rgi_df_2km,on='rgi_id',how='inner')

In [103]:
climate_historical_1980_2000['prcp'] = climate_historical_1980_2000['prcp']/10

In [104]:
climate_historical_1980_2000

,time,prcp,temp,rgi_id,Area
0,1980-01-01_1990-01-01,249.70903,-14.657376,RGI60-13.00062,2.222
1,1990-01-01_2000-01-01,235.78118,-13.965865,RGI60-13.00062,2.222
2,1980-01-01_1990-01-01,251.52350,-16.065098,RGI60-13.00093,2.019
3,1990-01-01_2000-01-01,240.99590,-15.468080,RGI60-13.00093,2.019
4,1980-01-01_1990-01-01,251.52350,-16.065098,RGI60-13.00130,2.439
...,...,...,...,...,...
16193,1990-01-01_2000-01-01,742.15840,-10.329490,RGI60-15.13104,22.704
16194,1980-01-01_1990-01-01,923.54110,-8.898493,RGI60-15.13116,7.427
16195,1990-01-01_2000-01-01,976.85900,-8.357633,RGI60-15.13116,7.427
16196,1980-01-01_1990-01-01,443.40664,-6.334142,RGI60-15.13117,6.788


In [47]:
# Make column with mean and difference 
ids = climate_historical_1980_2000['rgi_id']
ids = ids.unique()
empty_list_id = []

empty_list_temp_diff = []
empty_list_temp_mean = []

empty_list_prcp_diff = []
empty_list_prcp_mean = []

for i in range(len(ids)):

    sel1980 =  climate_historical_1980_2000[(climate_historical_1980_2000['rgi_id'] == ids[i]) & (climate_historical_1980_2000['time'] == '1980-01-01_1990-01-01')]
    sel1990 =  climate_historical_1980_2000[(climate_historical_1980_2000['rgi_id'] == ids[i]) & (climate_historical_1980_2000['time'] == '1990-01-01_2000-01-01')]
    
    value_temp_diff = sel1990['temp'].iloc[0] - sel1980['temp'].iloc[0]
    value_temp_mean = (sel1990['temp'].iloc[0] + sel1980['temp'].iloc[0])/2
    
    value_prcp_diff = sel1990['prcp'].iloc[0] - sel1980['prcp'].iloc[0]
    value_prcp_mean = (sel1990['prcp'].iloc[0] + sel1980['prcp'].iloc[0])/2
   
    empty_list_temp_diff.append(value_temp_diff)
    empty_list_temp_mean.append(value_temp_mean)
    empty_list_prcp_diff.append(value_prcp_diff)
    empty_list_prcp_mean.append(value_prcp_mean)
    
    id_take = ids[i]
    empty_list_id.append(id_take)
   
    
d = {'rgi_id':empty_list_id,'temp_diff_1980-2000':empty_list_temp_diff,'temp_mean_1980-2000':empty_list_temp_mean,'prcp_diff_1980_2000':empty_list_prcp_diff,'prcp_mean_1980_2000':empty_list_prcp_mean}
temp_prcp_1980_2000 = pd.DataFrame(d)

In [105]:
temp_prcp_1980_2000

,rgi_id,temp_diff_1980-2000,temp_mean_1980-2000,prcp_diff_1980_2000,prcp_mean_1980_2000
0,RGI60-13.00062,0.691511,-14.311621,-13.92785,242.745105
1,RGI60-13.00093,0.597017,-15.766589,-10.52760,246.259700
2,RGI60-13.00130,0.597017,-15.766589,-10.52760,246.259700
3,RGI60-13.00135,0.597017,-15.766589,-10.52760,246.259700
4,RGI60-13.00137,0.597017,-15.766589,-10.52760,246.259700
...,...,...,...,...,...
8094,RGI60-15.13092,0.316099,-2.823534,-68.40080,1335.975300
8095,RGI60-15.13097,0.648254,-9.576215,-7.38325,1086.739575
8096,RGI60-15.13104,0.561388,-10.610184,28.29330,728.011750
8097,RGI60-15.13116,0.540860,-8.628063,53.31790,950.200050


In [106]:
# Merge df with climate data 1980 - 2000 on rgi_id
df2000_2020 = df2000_2020.merge(temp_prcp_1980_2000,on='rgi_id',how='inner')


## Load SMB data between 2000 and 2020

In [107]:
hugonnet_0020 = pd.read_csv('C:/Users/david/Documents/Thesis/Data/hugonnet_2021_ds_rgi60_pergla_rates_10_20_worldwide.csv')
hugonnet_0020 = hugonnet_0020[hugonnet_0020["period"].str.contains("2000-01-01_2020-01-01")==True]
hugonnet_0020 = hugonnet_0020.rename(columns={"rgiid": "rgi_id"})
hugonnet_0020 = hugonnet_0020[hugonnet_0020["period"].str.contains("2000-01-01_2020-01-01")==True]
hugonnet_0020 = hugonnet_0020.rename(columns={"rgiid": "rgi_id"})
hugonnet_0020 = hugonnet_0020[['rgi_id','period','dmdtda','dvoldt']]
df2000_2020 = pd.merge(hugonnet_0020,df2000_2020, on=['rgi_id'], how = 'inner')

In [109]:
df2000_2020 = df2000_2020.drop(['period_y','dmdtda_y','prcp','temp','period_x', 'dvoldt_y'], axis=1)
df2000_2020 = df2000_2020.rename(columns={'dmdtda_x':'dmdtda', 'dvoldt_x':'dvoldt'})
df2000_2020 = df2000_2020.drop_duplicates().reset_index(drop=True)

In [113]:
df2000_2020 = df2000_2020.drop_duplicates().reset_index(drop=True)
df2000_2020


,rgi_id,dmdtda,dvoldt,tstar_aar,tstar_ela_h,avg_ice_thickness,ice_volume,glacial_lake,Slope,Aspect,...,debris_vol_ela_p,velocity_mean,temp_diff_2000_2020,temp_mean_2000_2020,prcp_diff_2000_2020,prcp_mean_2000_2020,temp_diff_1980-2000,temp_mean_1980-2000,prcp_diff_1980_2000,prcp_mean_1980_2000
0,RGI60-13.00062,0.0497,129921.0,0.622839,5777.281395,47.280240,0.105057,0.0,21.5,358.0,...,0.063558,0.646472,0.152752,-13.178201,-9.12109,261.668895,0.691511,-14.311621,-13.92785,242.745105
1,RGI60-13.00093,0.0660,156729.0,0.349641,5766.503904,31.963816,0.064535,0.0,25.6,50.0,...,0.384015,NaN,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700
2,RGI60-13.00130,0.1536,440650.0,0.482537,5864.392600,36.834835,0.089840,0.0,26.4,346.0,...,0.018882,1.383716,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700
3,RGI60-13.00135,0.0250,220540.0,0.673952,5885.474032,94.934830,0.710967,0.0,16.4,293.0,...,0.519112,1.272766,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700
4,RGI60-13.00137,0.0468,122568.0,0.541454,6039.362005,55.205690,0.122777,0.0,20.6,143.0,...,0.313242,0.564855,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8094,RGI60-15.13092,-0.9873,-5365263.0,0.544967,5132.936887,46.040108,0.212659,0.0,21.2,165.0,...,0.517255,2.397869,0.624130,-2.584252,-18.12140,1209.581000,0.316099,-2.823534,-68.40080,1335.975300
8095,RGI60-15.13097,-0.3773,-1605127.0,0.587474,5643.597799,75.617004,0.273431,0.0,8.8,20.0,...,0.003631,1.604927,0.146501,-9.092999,16.70860,1072.780300,0.648254,-9.576215,-7.38325,1086.739575
8096,RGI60-15.13104,-0.4842,-12932908.0,0.551624,5805.561280,109.057915,2.476051,1.0,12.2,341.0,...,13.621900,4.724451,0.073012,-10.079212,5.52163,700.782545,0.561388,-10.610184,28.29330,728.011750
8097,RGI60-15.13116,-0.8532,-7454719.0,0.537703,5613.211517,90.726250,0.673824,0.0,8.9,330.0,...,0.523140,6.024705,0.315175,-8.041431,26.31840,951.186000,0.540860,-8.628063,53.31790,950.200050


## Add O2 regions

In [114]:
regions = df2000_2020[['rgi_id','O2Region']]
regions['O1Region'] = df2000_2020['rgi_id'].str[6:8]
regions['O2Region'] = regions['O2Region'].astype(str)
type(regions['O2Region'][0])

regions['Region'] = regions['O1Region']  + '_' + regions['O2Region']
regions

df2000_2020 = pd.merge(df2000_2020, regions[['rgi_id','Region','O1Region']], on=['rgi_id'])
df2000_2020

C:\Users\david\AppData\Local\Temp\ipykernel_15180\2216749014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regions['O1Region'] = df2000_2020['rgi_id'].str[6:8]
C:\Users\david\AppData\Local\Temp\ipykernel_15180\2216749014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regions['O2Region'] = regions['O2Region'].astype(str)
C:\Users\david\AppData\Local\Temp\ipykernel_15180\2216749014.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,rgi_id,dmdtda,dvoldt,tstar_aar,tstar_ela_h,avg_ice_thickness,ice_volume,glacial_lake,Slope,Aspect,...,temp_diff_2000_2020,temp_mean_2000_2020,prcp_diff_2000_2020,prcp_mean_2000_2020,temp_diff_1980-2000,temp_mean_1980-2000,prcp_diff_1980_2000,prcp_mean_1980_2000,Region,O1Region
0,RGI60-13.00062,0.0497,129921.0,0.622839,5777.281395,47.280240,0.105057,0.0,21.5,358.0,...,0.152752,-13.178201,-9.12109,261.668895,0.691511,-14.311621,-13.92785,242.745105,13_5,13
1,RGI60-13.00093,0.0660,156729.0,0.349641,5766.503904,31.963816,0.064535,0.0,25.6,50.0,...,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700,13_5,13
2,RGI60-13.00130,0.1536,440650.0,0.482537,5864.392600,36.834835,0.089840,0.0,26.4,346.0,...,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700,13_5,13
3,RGI60-13.00135,0.0250,220540.0,0.673952,5885.474032,94.934830,0.710967,0.0,16.4,293.0,...,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700,13_5,13
4,RGI60-13.00137,0.0468,122568.0,0.541454,6039.362005,55.205690,0.122777,0.0,20.6,143.0,...,0.123990,-14.701116,1.19615,247.730395,0.597017,-15.766589,-10.52760,246.259700,13_5,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8094,RGI60-15.13092,-0.9873,-5365263.0,0.544967,5132.936887,46.040108,0.212659,0.0,21.2,165.0,...,0.624130,-2.584252,-18.12140,1209.581000,0.316099,-2.823534,-68.40080,1335.975300,15_3,15
8095,RGI60-15.13097,-0.3773,-1605127.0,0.587474,5643.597799,75.617004,0.273431,0.0,8.8,20.0,...,0.146501,-9.092999,16.70860,1072.780300,0.648254,-9.576215,-7.38325,1086.739575,15_1,15
8096,RGI60-15.13104,-0.4842,-12932908.0,0.551624,5805.561280,109.057915,2.476051,1.0,12.2,341.0,...,0.073012,-10.079212,5.52163,700.782545,0.561388,-10.610184,28.29330,728.011750,15_1,15
8097,RGI60-15.13116,-0.8532,-7454719.0,0.537703,5613.211517,90.726250,0.673824,0.0,8.9,330.0,...,0.315175,-8.041431,26.31840,951.186000,0.540860,-8.628063,53.31790,950.200050,15_1,15


In [116]:
# Export dataframe
df2000_2020.to_csv('df2000_2020.csv')